In [2]:
import os
from dotenv import load_dotenv
from slack_sdk import WebClient

load_dotenv()

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

# 슬랙 봇 토큰
SLACK_BOT_TOKEN = SLACK_BOT_TOKEN

In [3]:
import time
import re
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

# Slack 클라이언트 초기화
client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

In [ ]:
# Slack message 전송하는 부분
def send_message(message: str, channel: str):
    try:
        client.chat_postMessage(channel=channel, text=message)
    except Exception as e:
        print(f"오류 발생: {e}")

# 웹 크롤링링
def crawl_with_selenium() -> str:
    driver = webdriver.Chrome()
    driver.get("https://coinness.com/article")
    time.sleep(5)  # JS 로딩 대기
    
    css = "#root > div > div.Wrap-sc-v065lx-0.hwmGSB > div > main > div.ContentContainer-sc-91rcal-0.jJHYjq > div.ArticleListContainer-sc-cj3rkv-0.fkxjqP"
    try:
        container = driver.find_element(By.CSS_SELECTOR, css)
        return container.text
    except Exception as e:
        print(f"[ERROR] 요소 찾기 실패: {e}")
        return ""
    finally:
        driver.quit()

# 파싱
def parse_news_text(raw_text: str) -> pd.DataFrame:
    lines = [line.strip() for line in raw_text.splitlines() if line.strip()]
    time_pattern = re.compile(r'^\d{2}:\d{2}$')

    articles = []
    current_time = None
    current_date = None
    current_title = None
    content_lines = []

    def save_article():
        if current_time and current_date and current_title:
            articles.append({
                "time": current_time,
                "date": current_date,
                "title": current_title,
                "content": "\n".join(content_lines).strip()
            })

    state = "idle"
    for line in lines:
        # (1) 새로운 시간(기사 시작 신호) 감지
        if time_pattern.match(line):
            save_article()
            current_time = line
            current_date = None
            current_title = None
            content_lines = []
            state = "got_time"
            continue
        
        # (2) got_time → 날짜
        if state == "got_time":
            current_date = line
            state = "got_date"
            continue
        
        # (3) got_date → 제목
        if state == "got_date":
            current_title = line
            state = "got_title"
            continue
        
        # (4) 나머지 줄은 본문
        content_lines.append(line)
        state = "collecting_content"

    # 마지막 기사 저장
    save_article()

    df = pd.DataFrame(articles, columns=["time", "date", "title", "content"])
    return df

def process_and_send(new_df: pd.DataFrame, csv_path: str, channel_name: str):
    if new_df.empty:
        print("[INFO] 새로 추출된 기사가 없습니다.")
        return
    
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
    else:
        existing_df = pd.DataFrame(columns=["time", "date", "title", "content"])
    
    # 이미 존재하는 기사 집합
    existing_signatures = {
        (row["time"], row["date"], row["title"], row["content"])
        for _, row in existing_df.iterrows()
    }
    
    new_articles_list = []
    for _, row in new_df.iterrows():
        sig = (row["time"], row["date"], row["title"], row["content"])
        if sig not in existing_signatures:
            new_articles_list.append(row)
            existing_signatures.add(sig)
    
    if not new_articles_list:
        print("[INFO] 추가된 새 기사가 없습니다.")
    else:
        # Slack 메시지 전송
        message_parts = []
        for row in new_articles_list:
            part = (
                f"[{row['time']}][{row['date']}]\n"
                f"{row['title']}\n"
                f"{row['content']}\n"
            )
            message_parts.append(part)
        final_msg = "".join(message_parts)
        send_message(final_msg, channel_name)
        
        print(f"[INFO] 새 기사 {len(new_articles_list)}건이 Slack에 전송되었습니다.")

        # (NEW) JSON 출력
        new_articles_df = pd.DataFrame(new_articles_list, columns=["time", "date", "title", "content"])
        json_output = convert_articles_to_json(new_articles_df)
        # print("[JSON OUTPUT] 새 기사들:", json_output)
        # 필요하다면 json_output을 파일에 쓰거나, 다른 API로 전송 가능
        # 이 아래에 프롬프트 전송할것들 작성해서 넣기 ex) 비트코인 현재가, 비트코인 1분/5분/15분봉, 뉴스컬럼럼
    
    # 최종 CSV 저장
    new_articles_df = pd.DataFrame(new_articles_list, columns=["time", "date", "title", "content"])
    final_df = pd.concat([existing_df, new_articles_df], ignore_index=True)
    final_df.drop_duplicates(subset=["time", "date", "title", "content"], inplace=True)
    final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[INFO] CSV 저장 완료 (총 {len(final_df)}건).")


def run_autoupdate(interval = 300, channel_name: str = CHANNEL_NAME):
    news_folder = "NEWS"
    os.makedirs(news_folder, exist_ok=True)
    csv_path = os.path.join(news_folder, "my_news.csv")
    
    print(f"[INFO] 자동 업데이트 시작 ({int(interval/60)}분 간격)")
    last_date = datetime.date.today()

    try:
        while True:
            now = datetime.datetime.now()
            today_date = now.date()
            
            # (A) 날짜가 바뀌었으면 CSV 초기화
            if today_date != last_date:
                if os.path.exists(csv_path):
                    os.remove(csv_path)
                    print(f"[INFO] 날짜 변경으로 CSV({csv_path})를 초기화했습니다.")
                last_date = today_date

            print(f"\n[INFO] 뉴스 확인 중... ({now.strftime('%Y-%m-%d %H:%M:%S')})")
            
            # (B) 크롤링 -> 파싱
            raw_text = crawl_with_selenium()
            df_new = parse_news_text(raw_text)
            
            # (C) 새 기사만 Slack 전송 후, CSV 병합 저장
            process_and_send(df_new, csv_path, channel_name)

            print(f"[INFO] {int(interval/60)}분 뒤 다음 업데이트 진행...")
            time.sleep(interval)
            
    except KeyboardInterrupt:
        print("[INFO] 자동 업데이트 종료 (KeyboardInterrupt).")

def convert_articles_to_json(df):
    """
    DataFrame을 JSON 문자열로 변환.
    (한글 깨짐 방지를 위해 ensure_ascii=False 사용)
    """
    # DataFrame -> list of dict
    data_list = df.to_dict(orient="records")

    # dict/list -> JSON
    json_str = json.dumps(data_list, ensure_ascii=False, indent=2)
    return json_str


# 메인 실행
if __name__ == "__main__":
    run_autoupdate(interval=600, channel_name=CHANNEL_NAME)


[INFO] 자동 업데이트 시작 (10분 간격)

[INFO] 뉴스 확인 중... (2025-01-23 09:47:13)
[INFO] 새 기사 4건이 Slack에 전송되었습니다.
[JSON OUTPUT] 새 기사들: [
  {
    "time": "09:32",
    "date": "2025년 1월 23일 목요일",
    "title": "美 상원의원, 올해 암호화폐 규제 프레임워크 개편 예고",
    "content": "미국 상원 은행·주택·도시 문제 위원회 의장을 맡은 팀 스콧 상원의원이 22일 열린 제119대 의회에서 미국 내 암호화폐 규제 재편 계획을 밝혔다. 스콧 의원은 이전 행정부의 암호화폐 규제 방식을 비판, 같은 실수를 되풀이해서는 안된다고 강조했다. 의회 연설에 나선 스콧 의원은 \\\"이전 행정부는 암호화폐"
  },
  {
    "time": "09:31",
    "date": "2025년 1월 23일 목요일",
    "title": "모건스탠리, 美 주식 집중 위험 경고…다각화 투자 강조",
    "content": "모건스탠리 웰스 매니지먼트는 최근 미국 주식시장의 과도한 밸류에이션과 정책 불확실성을 우려하며 고객들에게 포트폴리오 다각화와 미국 주식 비중 축소를 권고했다.22일(현지시간) 미국 대형주 벤치마크인 스탠더드앤드푸어스(S&P) 500지수가 사상 최고치를 기록한 가운데 모건스탠리는 미국 주식의 가격이 지나치"
  },
  {
    "time": "09:29",
    "date": "2025년 1월 23일 목요일",
    "title": "[코인 시황] 비트코인 추가 호재 기다리며 10.5K 근방서 등락",
    "content": "[서울=뉴스핌] 고인원 기자= 크립토 태스크포스(TF) 출범 소식에 반등했던 비트코인 가격은 이후 구체적인 호재가 나오지 않자 다시 하락세로 전환하며 지지부진한 모습을 보이고 있다. 코인데스크에 따르면 비트코인 가격은 한국 시간 기준 23일 오전 9시 현재 24시간"
  },


In [4]:
# 빗썸 비트코인 현재가 조회
import python_bithumb

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

In [10]:
access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = python_bithumb.Bithumb(access_key, secret_key)

krw_balance = bithumb.get_balance("KRW")

# 다중 현재가 조회
price = python_bithumb.get_current_price(["KRW-BTC"])


# 시장가 매수 주문
# order_info = bithumb.buy_market_order("KRW-BTC", 10000)

# 주문 가능 조회
chance_info = bithumb.get_order_chance("KRW-BTC")
print(chance_info)

print(price)
print(krw_balance,"원")



{'bid_fee': '0.0025', 'ask_fee': '0.0025', 'maker_bid_fee': '0.0025', 'maker_ask_fee': '0.0025', 'market': {'id': 'KRW-BTC', 'name': 'BTC/KRW', 'order_types': ['limit'], 'order_sides': ['ask', 'bid'], 'bid_types': ['limit', 'price'], 'ask_types': ['limit', 'market'], 'bid': {'currency': 'KRW', 'min_total': '5000'}, 'ask': {'currency': 'BTC', 'min_total': '5000'}, 'max_total': '1000000000', 'state': 'active'}, 'bid_account': {'currency': 'KRW', 'balance': '10000', 'locked': '0', 'avg_buy_price': '0', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}, 'ask_account': {'currency': 'BTC', 'balance': '0', 'locked': '0', 'avg_buy_price': '0', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}}
154088000.0
10000.0 원
